# Tutorial: Analysis

In [1]:
from datetime import datetime
from straklip import config, input_tables
from straklip.steps import analysis
from straklip.stralog import getLogger
import os
import pkg_resources as pkg

First, we need to initialize the logger here.

In [2]:
if 'SHARED_LOG_FILE' not in os.environ:
    os.environ['SHARED_LOG_FILE'] = f'straklip_{datetime.now().strftime("%Y-%m-%d_%H%M")}.log'

getLogger('straklip', setup=True, logfile=os.environ['SHARED_LOG_FILE'],debu=False,
          configfile=pkg.resource_filename('straklip', './config/logging.yaml'))

<Logger straklip (DEBUG)>

For this tutorial we will start form the previously generated dataframes from the pipeline.
Note that in the pipe.yaml, we use unq_ids_list: [52] so the pipeline will run the reduction only on this target, to showcase the pipelien and save time and space.

In [3]:
pipe_cfg='/Users/gstrampelli/StraKLIP/Tutorial/pipeline_logs/pipe.yaml' #or where these files are
data_cfg='/Users/gstrampelli/StraKLIP/Tutorial/pipeline_logs/data.yaml'
pipe_cfg = config.configure_pipeline(pipe_cfg,pipe_cfg=pipe_cfg,data_cfg=data_cfg,dt_string=datetime.now().strftime("%d/%m/%Y %H:%M:%S"))
data_cfg = config.configure_data(data_cfg,pipe_cfg)

2025-06-05 14:16:12 config                      :INFO     (configure_pipeline:72[pid=77725]) 
StraKLIP pipeline started at date and time: 05/06/2025 14:16:12
Pipe_cfg: /Users/gstrampelli/StraKLIP/Tutorial/pipeline_logs/pipe.yaml
Data_cfg: /Users/gstrampelli/StraKLIP/Tutorial/pipeline_logs/data.yaml

2025-06-05 14:16:12 config                      :INFO     (configure_data:153[pid=77725]) Validation of default labels and data successful!


Once the "pipe_cfg" and the "data_cfg" are configuration, we can load pre-existing dataframes.

In this case we use "skip_originals" True and "load" True to tell the pipeline tom not look for inpout catalogs, but for the dataframe already generated by the pipeline.

NOTE: for this tutorial, we are forcing the pipeline to work on only the "unq_id" 52 to speed up the process, as follow (you can achieve the same result by changing the "unq_ids_list" option in the "pipe.yaml"):

In [4]:
pipe_cfg.unq_ids_list = [52]


In [5]:
dataset = input_tables.Tables(data_cfg, pipe_cfg, skip_originals=True)
DF = config.configure_dataframe(dataset,load=True)

2025-06-05 14:16:14 config                      :WARNING  (configure_dataframe:226[pid=77725]) get_Av_dict currently only supports VEGAmag system. Please provide your own set of AVs if in a differest system as AVs : {ext: {mag_filter : value}} in the data.yaml under target
2025-06-05 14:16:14 ancillary                   :INFO     (get_Av_dict:425[pid=77725]) before dust, V =  0.0 mag(VEGA)
2025-06-05 14:16:14 ancillary                   :INFO     (get_Av_dict:426[pid=77725]) after dust, V = 1.0146 mag(VEGA)
2025-06-05 14:16:14 ancillary                   :INFO     (get_Av_dict:445[pid=77725]) Av = 1.0146 mag


2025-06-05 14:16:14 ancillary                   :INFO     (get_Av_dict:489[pid=77725]) AV=0 wfc3,uvis2,f814w 0.0 mag(VEGA)
2025-06-05 14:16:14 ancillary                   :INFO     (get_Av_dict:490[pid=77725]) AV=1 wfc3,uvis2,f814w 0.6094 mag
2025-06-05 14:16:14 ancillary                   :INFO     (get_Av_dict:489[pid=77725]) AV=0 wfc3,uvis2,f850lp 0.0 mag(VEGA)
2025-06-05 14:16:14 ancillary                   :INFO     (get_Av_dict:490[pid=77725]) AV=1 wfc3,uvis2,f850lp 0.4694 mag
2025-06-05 14:16:14 config                      :INFO     (configure_dataframe:272[pid=77725]) Fetching dataframes from /Users/gstrampelli/StraKLIP/Tutorial/out


In [6]:
DF.keys

['unq_targets',
 'crossmatch_ids',
 'mvs_targets',
 'mvs_candidates',
 'unq_candidates']

The next step wil perform a more in depth analysis of each candidate. Manipulating the options in the "pipe.yaml" under the "analysis" block we can perform raw contrast curves, calibrated contrast curves, the extraction of a candidate though forward modeling and MCMC fitting, and conversion of contrast to masses using user provided interpolator that will transform (mag,age) -> (mass), (mag,age) -> (teff), (mag,age) -> (logR).

This interpolator should be assemble as a series of dictionaries such as interp = {filter: {'logmass': (mag,age) -> (mass), 'teff': (mag,age) -> (teff), 'logR': (mag,age) -> (logR)}}

In the following we force the pipeline to just run  the extract_candidate step, and we ensure that all the other step of the analysis are turned off.

In [7]:
pipe_cfg.analysis['steps']['extract_candidate']=True
pipe_cfg.analysis['steps']['contrast_curves']=False
pipe_cfg.analysis['steps']['cal_contrast_curves']=False
pipe_cfg.analysis['steps']['mass_sensitivity_curves']=False

In [8]:
analysis.run({'DF': DF, 'dataset': dataset})

2025-06-05 14:16:17 steps.analysis              :INFO     (run:1485[pid=77725]) Running Analysis step
2025-06-05 14:16:17 straklip.config             :INFO     (make_paths:117[pid=77725]) "/Users/gstrampelli/StraKLIP/Tutorial/out/analysis/" exists, and will not be created.
2025-06-05 14:16:17 steps.analysis              :INFO     (setup_DATASET:93[pid=77725]) Setting up Dataset for observation
2025-06-05 14:16:17 steps.analysis              :INFO     (generate_psflib:114[pid=77725]) Generating PSF library
2025-06-05 14:16:17 steps.analysis              :INFO     (candidate_extraction:929[pid=77725]) Extracting candidate from: ['/Users/gstrampelli/StraKLIP/Tutorial/out/mvs_tiles/f814w/tile_ID52.fits']
2025-06-05 14:16:17 steps.analysis              :INFO     (run_FMAstrometry:708[pid=77725]) Running forward modeling


/opt/miniconda3/envs/straKLIP/lib/python3.11/site-packages/pyklip/fm.py:678: RuntimeWarning: invalid value encountered in divide
  perturb_mag = np.abs(quad_perturb/linear_perturb)



2025-06-05 14:16:22 steps.analysis              :INFO     (run_FMAstrometry:794[pid=77725])  contrast: 0.499, error: [ 0.009 -0.009]
2025-06-05 14:16:22 steps.analysis              :INFO     (fit_astrometry:880[pid=77725]) Saving MCMC plots in: /Users/gstrampelli/StraKLIP/Tutorial/out/analysis/ID52/extracted_candidate/testPSF4extraction/
2025-06-05 14:16:22 steps.analysis              :INFO     (candidate_extraction:944[pid=77725]) Saving candidate dictionary to file: /Users/gstrampelli/StraKLIP/Tutorial/out/analysis/ID52/extracted_candidate/testPSF4extraction//f814w_cand_1_extracted.yaml
2025-06-05 14:16:22 steps.analysis              :INFO     (run_FMAstrometry:708[pid=77725]) Running forward modeling


/opt/miniconda3/envs/straKLIP/lib/python3.11/site-packages/pyklip/fm.py:678: RuntimeWarning: invalid value encountered in divide
  perturb_mag = np.abs(quad_perturb/linear_perturb)



2025-06-05 14:16:27 steps.analysis              :INFO     (run_FMAstrometry:794[pid=77725])  contrast: 0.654, error: [ 0.011 -0.009]
2025-06-05 14:16:27 steps.analysis              :INFO     (fit_astrometry:880[pid=77725]) Saving MCMC plots in: /Users/gstrampelli/StraKLIP/Tutorial/out/analysis/ID52/extracted_candidate/testPSF4extraction/
2025-06-05 14:16:27 steps.analysis              :INFO     (candidate_extraction:944[pid=77725]) Saving candidate dictionary to file: /Users/gstrampelli/StraKLIP/Tutorial/out/analysis/ID52/extracted_candidate/testPSF4extraction//f814w_cand_2_extracted.yaml
2025-06-05 14:16:27 steps.analysis              :INFO     (run_FMAstrometry:708[pid=77725]) Running forward modeling


...

/opt/miniconda3/envs/straKLIP/lib/python3.11/site-packages/pyklip/fm.py:678: RuntimeWarning: invalid value encountered in divide
  perturb_mag = np.abs(quad_perturb/linear_perturb)



2025-06-05 14:20:33 steps.analysis              :INFO     (run_FMAstrometry:794[pid=77725])  contrast: 0.629, error: [ 0.01  -0.011]
2025-06-05 14:20:33 steps.analysis              :INFO     (fit_astrometry:880[pid=77725]) Saving MCMC plots in: /Users/gstrampelli/StraKLIP/Tutorial/out/analysis/ID52/extracted_candidate/testPSF4extraction/
2025-06-05 14:20:34 steps.analysis              :INFO     (candidate_extraction:944[pid=77725]) Saving candidate dictionary to file: /Users/gstrampelli/StraKLIP/Tutorial/out/analysis/ID52/extracted_candidate/testPSF4extraction//f814w_cand_50_extracted.yaml
2025-06-05 14:20:34 steps.analysis              :INFO     (run_FMAstrometry:708[pid=77725]) Running forward modeling


/opt/miniconda3/envs/straKLIP/lib/python3.11/site-packages/pyklip/fm.py:678: RuntimeWarning: invalid value encountered in divide
  perturb_mag = np.abs(quad_perturb/linear_perturb)



2025-06-05 14:20:38 steps.analysis              :INFO     (run_FMAstrometry:794[pid=77725])  contrast: 0.581, error: [ 0.01 -0.01]
2025-06-05 14:20:38 steps.analysis              :INFO     (fit_astrometry:880[pid=77725]) Saving MCMC plots in: /Users/gstrampelli/StraKLIP/Tutorial/out/analysis/ID52/extracted_candidate/testPSF4extraction/
2025-06-05 14:20:40 steps.analysis              :INFO     (candidate_extraction:944[pid=77725]) Saving candidate dictionary to file: /Users/gstrampelli/StraKLIP/Tutorial/out/analysis/ID52/extracted_candidate/testPSF4extraction//f814w_cand_51_extracted.yaml
2025-06-05 14:20:40 steps.analysis              :INFO     (candidate_extraction:998[pid=77725]) Loading isochrones interpolator dictionary from file: /Users/gstrampelli/PycharmProjects/NGC1976/database/interpolated_isochrones.pck


KeyError: 'f814w'